# CFFI ABI Mode Examples for Windows

This notebook demonstrates **Windows-compatible CFFI ABI examples** that solve the common issue where `ffi.dlopen(None)` doesn't work on Windows.

## Background

The original CFFI documentation example:
```python
from cffi import FFI
ffi = FFI()
ffi.cdef("int printf(const char *format, ...);")
C = ffi.dlopen(None)  # ❌ This doesn't work on Windows!
C.printf(b"Hello world\n")
```

## The Windows Solution

Instead of using `ffi.dlopen(None)`, we load specific Windows DLLs that are guaranteed to be available:

### 🔧 Example 1: Using `msvcrt.dll` (Microsoft C Runtime)
- **Functions**: `printf()`, `puts()`, `getenv()`, `_putch()`
- **Use case**: Standard C library functions
- **Availability**: Present on all Windows systems

### 🪟 Example 2: Using `kernel32.dll` (Windows API)
- **Functions**: `GetTickCount()`, `GetCurrentProcessId()`, `GetComputerNameA()`
- **Use case**: Windows-specific system information
- **Availability**: Core Windows system library

## Key Advantages

✅ **ABI Mode**: No C compiler required  
✅ **Windows Compatible**: Works reliably on all Windows versions  
✅ **Fast**: Direct function calls without compilation overhead  
✅ **Portable**: Uses system DLLs available everywhere  

## Important Notes

⚠️ **String Handling**: C functions expect `bytes` objects, not Python strings  
⚠️ **Memory Management**: Use `ffi.new()` to create C-compatible data structures  
⚠️ **Return Values**: C functions return their native types (int, pointers, etc.)  

---

## Example 1: Microsoft C Runtime Functions

This first example uses `msvcrt.dll` (Microsoft Visual C++ Runtime) which provides standard C library functions. This is the closest equivalent to the original documentation example that works reliably on Windows.

**Key concepts:**
- **`printf()`**: Formatted string output (similar to the original example)
- **`puts()`**: Simple string output function
- **`getenv()`**: Environment variable access
- **String conversion**: Converting between Python bytes and C strings

In [6]:
# Windows-compatible CFFI ABI example using msvcrt.dll (C runtime library)
from cffi import FFI

ffi = FFI()

# Define C functions from msvcrt (Microsoft Visual C++ Runtime)
ffi.cdef("""
    int printf(const char *format, ...);
    int puts(const char *string);
    char* getenv(const char *name);
    int _putch(int c);
""")

# Load the Microsoft C Runtime Library (available on all Windows systems)
C = ffi.dlopen("msvcrt.dll")

print("=== Windows CFFI ABI Example ===")

# Example 1: Using printf (similar to the original example)
print("\n1. Using printf:")
arg = ffi.new("char[]", b"Windows")
result = C.printf(b"Hello from CFFI on %s!\n", arg)
print(f"printf returned: {result}")

# Example 2: Using puts (simpler string output)
print("\n2. Using puts:")
message = ffi.new("char[]", b"This is a test message from puts()")
result = C.puts(message)
print(f"puts returned: {result}")

# Example 3: Using getenv to read environment variable
print("\n3. Reading environment variable:")
var_name = ffi.new("char[]", b"PATH")
path_ptr = C.getenv(var_name)
if path_ptr != ffi.NULL:
    # Convert C string to Python string (first 100 chars to avoid flooding output)
    path_str = ffi.string(path_ptr, 100).decode('utf-8', errors='ignore')
    print(f"PATH (first 100 chars): {path_str}...")
else:
    print("PATH environment variable not found")

print("\n=== Example completed successfully! ===")

=== Windows CFFI ABI Example ===

1. Using printf:
printf returned: 28

2. Using puts:
puts returned: 0

3. Reading environment variable:
PATH (first 100 chars): c:\Users\htdun\anaconda3\envs\temp;C:\Users\htdun\anaconda3\envs\temp;C:\Users\htdun\anaconda3\envs\...

=== Example completed successfully! ===


## Example 2: Windows API Functions

The second example demonstrates accessing Windows-specific API functions from `kernel32.dll`. This shows how CFFI can be used to call native Windows system functions directly from Python.

**Key concepts:**
- **System uptime**: `GetTickCount()` returns milliseconds since system start
- **Process information**: `GetCurrentProcessId()` gets the current Python process ID  
- **System information**: `GetComputerNameA()` retrieves the computer name
- **Buffer management**: Shows how to work with C buffers and pointers

In [7]:
# Alternative Windows example using kernel32.dll for Windows API functions
from cffi import FFI

ffi = FFI()

# Define Windows API functions from kernel32.dll
ffi.cdef("""
    unsigned long GetTickCount();
    void GetSystemTime(void* lpSystemTime);
    unsigned long GetCurrentProcessId();
    int GetComputerNameA(char* lpBuffer, unsigned long* lpnSize);
""")

# Load kernel32.dll (Windows system library)
kernel32 = ffi.dlopen("kernel32.dll")

print("=== Windows API CFFI Example ===")

# Example 1: Get system uptime in milliseconds
print("\n1. System uptime:")
uptime = kernel32.GetTickCount()
uptime_seconds = uptime / 1000
uptime_minutes = uptime_seconds / 60
print(f"System uptime: {uptime} ms ({uptime_minutes:.1f} minutes)")

# Example 2: Get current process ID
print("\n2. Current process ID:")
process_id = kernel32.GetCurrentProcessId()
print(f"Current Python process ID: {process_id}")

# Example 3: Get computer name
print("\n3. Computer name:")
buffer_size = ffi.new("unsigned long*", 256)  # Buffer size pointer
buffer = ffi.new("char[]", 256)               # Character buffer

success = kernel32.GetComputerNameA(buffer, buffer_size)
if success:
    computer_name = ffi.string(buffer).decode('utf-8')
    print(f"Computer name: {computer_name}")
    print(f"Name length: {buffer_size[0]} characters")
else:
    print("Failed to get computer name")

print("\n=== Windows API example completed! ===")

=== Windows API CFFI Example ===

1. System uptime:
System uptime: 88518015 ms (1475.3 minutes)

2. Current process ID:
Current Python process ID: 3296

3. Computer name:
Computer name: DUDAKA
Name length: 6 characters

=== Windows API example completed! ===


## Summary

These examples demonstrate how to use CFFI in **ABI mode** on Windows by loading specific system DLLs instead of trying to access the entire C namespace with `ffi.dlopen(None)`.

### What we learned:

1. **Windows Compatibility**: Use `msvcrt.dll` and `kernel32.dll` instead of `ffi.dlopen(None)`
2. **Function Declarations**: Use `ffi.cdef()` to declare C function signatures
3. **Memory Management**: Use `ffi.new()` to create C-compatible data structures
4. **String Handling**: Convert between Python strings and C strings using `bytes` and `ffi.string()`
5. **Return Values**: Handle C function return values and error checking

### Next Steps:

- **API Mode**: For production use, consider using CFFI's API mode with a C compiler for better performance and safety
- **Error Handling**: Add proper error checking for C function calls
- **Custom DLLs**: Apply these techniques to load and use your own compiled C libraries
- **Structure Handling**: Explore using C structures with `ffi.new("struct_name*")`

### Resources:

- [CFFI Documentation](https://cffi.readthedocs.io/)
- [Windows API Reference](https://docs.microsoft.com/en-us/windows/win32/api/)
- [Microsoft C Runtime Reference](https://docs.microsoft.com/en-us/cpp/c-runtime-library/)